In [1]:
# Importing the required libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df=pd.read_csv('/content/isear.csv',header=None)
# Rename columns and reorder them
df.columns = ['emotion', 'text']
df.head()

,emotion,text
0,joy,[ On days when I feel close to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7652 entries, 0 to 7651
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  7652 non-null   object
 1   text     7652 non-null   object
dtypes: object(2)
memory usage: 119.7+ KB


In [4]:
noResponse = df[df['text'] == '[ No response.]'].index
df.drop(noResponse, inplace=True)


In [5]:
#select the text column
emotion_arr=df['text']

#first row
emotion_arr[0]

'[ On days when I feel close to my partner and other friends.  \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.]'

In [6]:
# loop over the text using word_tokenize
emotion_arr=[word_tokenize(sent) for sent in emotion_arr]
print(emotion_arr[0])


['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']']


In [7]:

# Defined a function padd in which each sentence length is fixed to 100.
# If length is less than 100 , then the word- '' is append
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('')
    return arr[:100]

# call the padd function for each sentence in emotion_arr
for i in range(len(emotion_arr)):
  emotion_arr[i]=padd(emotion_arr[i])

In [8]:
print(emotion_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [12]:
!pip install opendatasets
!pip install pandas

In [13]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/watts2/glove6b50dtxt?resource=download")
#"username":"palak2802","key":"9283fc99344ac05945cb97061bd7dbd5"

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: palak2802
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/watts2/glove6b50dtxt


100%|██████████| 67.7M/67.7M [00:00<00:00, 83.8MB/s]


In [16]:
vocab_f ='/content/glove6b50dtxt/glove.6B.50d.txt'

In [17]:
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# the embedding index of word 'happy'
embeddings_index['happy']

array([ 0.092086,  0.2571  , -0.58693 , -0.37029 ,  1.0828  , -0.55466 ,
       -0.78142 ,  0.58696 , -0.58714 ,  0.46318 , -0.11267 ,  0.2606  ,
       -0.26928 , -0.072466,  1.247   ,  0.30571 ,  0.56731 ,  0.30509 ,
       -0.050312, -0.64443 , -0.54513 ,  0.86429 ,  0.20914 ,  0.56334 ,
        1.1228  , -1.0516  , -0.78105 ,  0.29656 ,  0.7261  , -0.61392 ,
        2.4225  ,  1.0142  , -0.17753 ,  0.4147  , -0.12966 , -0.47064 ,
        0.3807  ,  0.16309 , -0.323   , -0.77899 , -0.42473 , -0.30826 ,
       -0.42242 ,  0.055069,  0.38267 ,  0.037415, -0.4302  , -0.39442 ,
        0.10511 ,  0.87286 ], dtype=float32)

In [18]:
# Embedding each word of the emotion_arr
embedded_emotion_arr = []

for each_sentence in emotion_arr:
    embedded_emotion_arr.append([])
    for word in each_sentence:
        if word.lower() in embeddings_index:
            embedded_emotion_arr[-1].append(embeddings_index[word.lower()])
        else:
            # if the word to be embedded is '' append 0 fifty times
            embedded_emotion_arr[-1].append([0]*50)

print(embedded_emotion_arr[0][0])


[-0.61201   0.98226   0.11539   0.014623  0.23873  -0.067035  0.30632
 -0.64742  -0.38517  -0.03691   0.094788  0.57631  -0.091557 -0.54825
  0.25255  -0.14759   0.13023   0.21658  -0.30623   0.30028  -0.23471
 -0.17927   0.9518    0.54258   0.31172  -0.51038  -0.65223  -0.48858
  0.13486  -0.40132   2.493    -0.38777  -0.26456  -0.49414  -0.3871
 -0.20983   0.82941  -0.46253   0.39549   0.014881  0.79485  -0.79958
 -0.16243   0.013862 -0.53536   0.52536   0.019818 -0.16353   0.30649
  0.81745 ]


In [19]:
# Perform one-hot encoding from sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(df['emotion']).reshape(-1,1)).toarray()
X=np.array(embedded_emotion_arr)


In [20]:

# Split into train and test
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [23]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense

In [26]:
# Defining the BiLSTM Model
def model(X, Y, input_size1, input_size2, output_size):
    m = Sequential()  # Create a Sequential model

    # Add a Bidirectional LSTM layer with 100 units
    m.add(Bidirectional(LSTM(100, input_shape=(input_size1, input_size2))))

    m.add(Dropout(0.5))  # Add a dropout layer with 50% dropout rate

    # Add a Dense (fully connected) layer with 'output_size' units and softmax activation
    m.add(Dense(output_size, activation='softmax'))

    # Compile the model with 'Adam' optimizer, 'categorical_crossentropy' loss, and 'accuracy' metric
    m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model on input data 'X' and target labels 'Y' for 32 epochs, using batches of size 128
    m.fit(X, Y, epochs=32, batch_size=128)

    return m  # Return the compiled and trained model


In [27]:
bilstmModel=model(X_train,Y_train,100,50,7)

Epoch 1/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.1761 - loss: 1.9448
Epoch 2/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3056 - loss: 1.7876
Epoch 3/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3803 - loss: 1.6502
Epoch 4/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4089 - loss: 1.5795
Epoch 5/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4157 - loss: 1.5588
Epoch 6/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4253 - loss: 1.5206
Epoch 7/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4602 - loss: 1.4594
Epoch 8/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4709 - loss: 1.4323
Epoch 9/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4846 - loss: 1.3858
Epoch 10/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4935 - loss: 1.3992
Epoch 11/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5136 - loss: 1.3428
Epoch 12/32
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy:

In [28]:
bilstmModel.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 200)                 │         120,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │           1,407 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 366,623 (1.40 MB)

 Trainable params: 122,207 (477.37 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 244,416 (954.75 KB)

In [29]:
#Testing the model
bilstmModel.evaluate(X_test,Y_test)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5247 - loss: 1.4064


[1.403466820716858, 0.5181518197059631]

In [30]:
# Preprocess input text for prediction
def preprocess_input_text(text):
    tokenized_text = word_tokenize(text)
    padded_text = padd(tokenized_text)

    embedded_text = []
    for word in padded_text:
        if word.lower() in embeddings_index:
            embedded_text.append(embeddings_index[word.lower()])
        else:
            embedded_text.append([0]*50)
    return np.array(embedded_text)

In [35]:
input_text = "I am feeling sad today."
processed_input = preprocess_input_text(input_text)


In [36]:
# Perform prediction using the loaded model
predicted_probs = bilstmModel.predict(np.array([processed_input]))
predicted_emotion_index = np.argmax(predicted_probs)

predicted_emotion_index

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


5

In [37]:
# Decode the predicted emotion
predicted_emotion = enc.categories_[0][predicted_emotion_index]

In [38]:
print("Predicted Emotion:", predicted_emotion)
print("Predicted Emotion Probabilities:", predicted_probs)

Predicted Emotion: sadness
Predicted Emotion Probabilities: [[1.1558315e-03 1.6843749e-03 1.2447110e-03 3.9781834e-04 7.5713195e-02
  9.1748476e-01 2.3193322e-03]]
